In [ ]:
!pip install sentencepiece
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install torch
!pip install sentencepiece
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292695 sha256=8e7de486a5e4b0cc8e13320ddd119ae726f88e8ec73502a2ad1c76aee5d5cac6
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp


In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-vobby568/kobert-tokenizer_6aa0f1801a0847dc9ed710b911c8846d
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-vobby568/kobert-tokenizer_6aa0f1801a0847dc9ed710b911c8846d
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=1780ad3919a52c4559c6b90b459c15f93e463751368f6ff68367084aa820af25
  Stored in directory: /tmp/pip-ephem-wheel-cache-3_24lnpb/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from PIL import Image
import numpy as np
import os
import glob

In [ ]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import glob
import os

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)

각 폴더에서 이미지와 텍스트 저장한 폴더 위치

In [ ]:
section_folders = glob.glob('/content/drive/MyDrive/NaverNews/*')
section_folders

['/content/drive/MyDrive/NaverNews/politics',
 '/content/drive/MyDrive/NaverNews/society',
 '/content/drive/MyDrive/NaverNews/science',
 '/content/drive/MyDrive/NaverNews/life',
 '/content/drive/MyDrive/NaverNews/world',
 '/content/drive/MyDrive/NaverNews/economy']

In [ ]:
root = '/content/drive/MyDrive/NaverNews'
sections = os.listdir(root)
sections

['politics', 'society', 'science', 'life', 'world', 'economy']

텍스트 위치

In [ ]:
text_paths = []
for folder, section in zip(section_folders, sections):
  text_path = folder + '/' + section + 'text1' + '.xlsx'
  text_paths.append(text_path)

text_paths

['/content/drive/MyDrive/NaverNews/politics/politicstext.xlsx',
 '/content/drive/MyDrive/NaverNews/society/societytext.xlsx',
 '/content/drive/MyDrive/NaverNews/science/sciencetext.xlsx',
 '/content/drive/MyDrive/NaverNews/life/lifetext.xlsx',
 '/content/drive/MyDrive/NaverNews/world/worldtext.xlsx',
 '/content/drive/MyDrive/NaverNews/economy/economytext.xlsx']

각 섹션별 인덱스 딕셔너리

In [ ]:
# 각 섹션별 사용할 데이터의 인덱스가 담겨 있는 딕셔너리

idx_dictionary = {}

for section, text_path in zip(sections, text_paths):
  text = pd.read_excel(text_path)
  idx_section = list(text['idx'])

  idx_dictionary[section] = idx_section

In [ ]:
np.random.seed(602)

cv_idx_dictionary = {}

for section in sections:
  cv_idx_section = list(np.random.choice(idx_dictionary[section], size = 1200, replace = False))
  cv_idx_section.sort()

  cv_idx_dictionary[section] = cv_idx_section

In [ ]:
cv_idx_dictionary.keys()

dict_keys(['politics', 'society', 'science', 'life', 'world', 'economy'])

In [ ]:
len(cv_idx_dictionary['economy'])

1200

In [ ]:
new_idx_dictionary = {}

for section in sections:
  new_idx_section = [i for i in idx_dictionary[section] if i not in cv_idx_dictionary[section]]

  new_idx_dictionary[section] = new_idx_section

In [ ]:
len(new_idx_dictionary['economy'])

688

label - y 페어

In [ ]:
# label_to_y = {section : idx for idx, section in enumerate(sections)}
# label_to_y

In [ ]:
label_to_y = {
    'politics': 0,
    'society': 1,
    'science': 2,
    'life': 3,
    'world': 4,
    'economy': 5
}
label_to_y

{'politics': 0,
 'society': 1,
 'science': 2,
 'life': 3,
 'world': 4,
 'economy': 5}

분석에 사용할 이미지 데이터 경로

In [ ]:
# 분석에 사용할 모든 데이터들의 경로를 불러온다.

cv_data = []
new_data = []

for section_folder in section_folders:

  # 각 섹션 이름 가져와서 인덱스랑 합하기
  section = os.path.basename(section_folder)
  section_textpath = section_folder + '/' + section + 'text1' + '.xlsx' #정제한걸로했음
  section_texts = pd.read_excel(section_textpath)


  cv_indicies = cv_idx_dictionary[section]
  new_indicies = new_idx_dictionary[section]

  cv_condition = section_texts['idx'].isin(cv_indicies)
  new_condition = section_texts['idx'].isin(new_indicies)

  cv_articles = section_texts.loc[cv_condition, 'title'] + ' ' + section_texts.loc[cv_condition, 'body']
  new_articles = section_texts.loc[new_condition, 'title'] + ' ' + section_texts.loc[new_condition, 'body']




  y = label_to_y[section]


  for cv_idx, cv_article in zip(cv_indicies, cv_articles):
    imgname = section + str(cv_idx) + '.jpg'
    imgpath = os.path.join(section_folder, imgname)

    data = []
    data.append(cv_idx)
    data.append(imgpath)
    data.append(cv_article)
    data.append(y)

    cv_data.append(data)


  for new_idx, new_article in zip(new_indicies, new_articles):
    imgname = section + str(new_idx) + '.jpg'
    imgpath = os.path.join(section_folder, imgname)

    data = []
    data.append(new_idx)
    data.append(imgpath)
    data.append(new_article)
    data.append(y)

    new_data.append(data)

['/content/drive/MyDrive/NaverNews/politics/politics0.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics1.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics2.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics3.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics4.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics5.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics6.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics7.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics8.jpg',
 '/content/drive/MyDrive/NaverNews/politics/politics9.jpg']

In [ ]:
df_cv_data = pd.DataFrame(cv_data)
df_cv_data.columns = ['idx', 'imgpath', 'article', 'y']

In [ ]:
df_new_data = pd.DataFrame(new_data)
df_new_data.columns = ['idx', 'imgpath', 'article', 'y']

데이터셋 정의하기

In [ ]:
# MobileNet image transform

train_imgtransform = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.Resize(256),
    #transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_imgtransform = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.Resize(256),
    #transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# BERT transform

class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [ ]:
class RoBaMFFusionDataset(Dataset):
  def __init__(self, dataset, imgtransform, bert_tokenizer, vocab, max_len, pad, pair):

    # for MobileNet
    self.imgpaths = dataset['imgpath']
    self.imgtransform = imgtransform


    # for KoBERT
    texttransform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
    self.kobertdata = dataset[['article', 'y']].values.tolist()
    self.sentences = [texttransform([i[0]]) for i in self.kobertdata]


    self.labels = [np.int32(i[1]) for i in self.kobertdata]


  def __getitem__(self, i):
    imgpath = self.imgpaths.iloc[i]

    img = Image.open(imgpath).convert('RGB')
    img = self.imgtransform(img)

    sentence = self.sentences[i]

    target = self.labels[i]

    ## sentence : (token_ids, valid_length, segment_ids) tuple
    return img, sentence, target

  def __len__(self):
    return len(self.labels)


In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# kobert 공식 git에 있는 get_kobert_model 선언
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj


In [ ]:
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

코버트 특징추출기

In [ ]:
class BERTFeatureExtractor(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 1024,   # Feature length : 1024
                 dr_rate = None,
                 params = None):
        super(BERTFeatureExtractor, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
kobert = BERTFeatureExtractor(bertmodel,  dr_rate = 0.5).to(device)

모바일넷v2 특징추출기

In [ ]:
mobilenetv2 = models.mobilenet_v2(weights = 'DEFAULT')

In [ ]:
# 이놈이 FusionModel에서 적용이 안 됨. 왜지?

#fc = nn.Sequential(
#    nn.Linear(1024, 1024),
#    nn.ReLU(),
#    nn.MaxPool1d(kernel_size=2, stride=2, padding=0),
#    nn.Linear(512, 512),
#    nn.ReLU(),
#    nn.MaxPool1d(kernel_size=2, stride=2, padding=0),
#    nn.Linear(256, 128),
#)

#mobilenetv2.fc = fc

In [ ]:
# 전이된 가중치는 고정

for i, (name, param) in enumerate(mobilenetv2.named_parameters()):
  param.requires_grad = False

  if i == 155:
    print('end')
    break

end


모델 정의 : Feature Fusion

In [ ]:
class FusionModel(nn.Module):
  def __init__(self, mobilenetv2, kobert):
    super(FusionModel, self).__init__()

    self.mobilenetv2 = mobilenetv2
    self.kobert = kobert

    self.fc_image = nn.Linear(1000, 512)
    self.fc_text = nn.Linear(1024, 512)

    self.classifier = nn.Sequential(
        nn.Linear(1024, 1024),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size = 2, stride = 2, padding = 0),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size = 2, stride = 2, padding = 0),
        nn.Linear(256, 6),
        nn.Softmax(1)
    )

  def forward(self, img, token_ids, valid_length, segment_ids):
        # 이미지 특징 추출
        image_feature = self.mobilenetv2(img)
        image_feature = self.fc_image(image_feature)

        # 텍스트 특징 추출
        text_feature = self.kobert(token_ids, valid_length, segment_ids)
        text_feature = self.fc_text(text_feature)

        # 두 특징을 결합
        x = torch.cat((image_feature, text_feature), dim=1)

        # 분류기 적용
        x = self.classifier(x)

        return x

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

Stratified K-Fold CV

In [ ]:
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 602)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
list_train_idx = []
list_test_idx = []

for train_idx, test_idx in cv.split(overall_data[['article', 'imgpath']], overall_data['y']):
  list_train_idx.append(train_idx)
  list_test_idx.append(test_idx)

In [ ]:
max_len = 100

In [ ]:
list_test_history = []

fold = 0

for train_idx, test_idx in zip(list_train_idx, list_test_idx):

  fold += 1

  warmup_ratio = 0.1
  num_epochs = 10     ### 실제 돌릴때 에포크 수정 ###
  max_grad_norm = 1
  log_interval = 200
  learning_rate =  5e-5


  bertmodel, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)
  kobert = BERTFeatureExtractor(bertmodel,  dr_rate = 0.5)
  mobilenetv2 = models.mobilenet_v2(weights = 'DEFAULT')

  model = FusionModel(mobilenetv2, kobert).to(device)

  train_data = df_cv_data.iloc[train_idx]
  test_data = df_cv_data.iloc[test_idx]

  train_dataset = RoBaMFFusionDataset(train_data, imgtransform = train_imgtransform, bert_tokenizer = tokenizer , vocab = vocab, max_len = max_len, pad = True, pair = False)
  test_dataset = RoBaMFFusionDataset(test_data, imgtransform = test_imgtransform, bert_tokenizer = tokenizer , vocab = vocab, max_len = max_len, pad = True, pair = False)

  train_dataloader = DataLoader(train_dataset, batch_size = 8, shuffle = True)
  test_dataloader = DataLoader(test_dataset, batch_size = 8, shuffle = True)


  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
      {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
  ]

  optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
  loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

  t_total = len(train_dataloader) * num_epochs
  warmup_step = int(t_total * warmup_ratio)

  scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)



  train_history = []
  test_history = []
  loss_history = []

  for e in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0
      model.train()
      for batch_id, (img, (token_ids, valid_length, segment_ids), label) in enumerate(tqdm(train_dataloader)):
          optimizer.zero_grad()

          img = img.to(device)
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length
          label = label.long().to(device)
          out = model(img, token_ids, valid_length, segment_ids)

          # print(label.shape, out.shape)
          loss = loss_fn(out, label)
          loss.backward()
          torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
          optimizer.step()
          scheduler.step()  # Update learning rate schedule
          train_acc += calc_accuracy(out, label)
          if batch_id % log_interval == 0:
              print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
              train_history.append(train_acc / (batch_id+1))
              loss_history.append(loss.data.cpu().numpy())
      print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
      # train_history.append(train_acc / (batch_id+1))


      # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
      # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
      model.eval()
      for batch_id, (img, (token_ids, valid_length, segment_ids), label) in enumerate(tqdm(test_dataloader)):
          img = img.to(device)
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length = valid_length
          label = label.long().to(device)
          out = model(img, token_ids, valid_length, segment_ids)
          test_acc += calc_accuracy(out, label)
      print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
      test_history.append(test_acc / (batch_id+1))


  # 모형 가중치 저장
  torch.save(model.state_dict(), f'/content/drive/MyDrive/Colab Notebooks/Machine Learning - Graduate/Model Weights/ArticleFusionWeight{fold}.pth')

  list_test_history.append(test_history)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/30 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.7911919355392456 train acc 0.375
epoch 1 train acc 0.175


  0%|          | 0/8 [00:00<?, ?it/s]

epoch 1 test acc 0.21875


...

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/30 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.7899487018585205 train acc 0.0
epoch 1 train acc 0.1875


  0%|          | 0/8 [00:00<?, ?it/s]

epoch 1 test acc 0.1875


New Data Prediction

In [ ]:
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)
kobert = BERTFeatureExtractor(bertmodel,  dr_rate = 0.5).to(device)
mobilenetv2 = models.mobilenet_v2(weights = 'DEFAULT')

model = FusionModel(mobilenetv2, kobert).to(device)

train_data = df_cv_data
test_data = df_new_data

train_dataset = RoBaMFFusionDataset(train_data, imgtransform = train_imgtransform, bert_tokenizer = tokenizer , vocab = vocab, max_len = max_len, pad = True, pair = False)
test_dataset = RoBaMFFusionDataset(test_data, imgtransform = test_imgtransform, bert_tokenizer = tokenizer , vocab = vocab, max_len = max_len, pad = True, pair = False)

train_dataloader = DataLoader(train_dataset, batch_size = 8, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 8, shuffle = True)


no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)



train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (img, (token_ids, valid_length, segment_ids), label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()

        img = img.to(device)
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(img, token_ids, valid_length, segment_ids)

        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    for batch_id, (img, (token_ids, valid_length, segment_ids), label) in enumerate(tqdm(test_dataloader)):
        img = img.to(device)
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(img, token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))